# Travelling Salesman:


## Document review(s)

# 1 - Abstract

In the corporate sector, there are many problems that decision-makers strive to resolve and offer the most productive working practices at the most profitble cost. One of the challenges we wanted to tackle in this project is delivery of goods.


The model developed for this research optimizes the cost (duration of travel) of shipping products from Venice to Milan while passing through other cities in Veneto.  This model includes specifications that are taken from the real world, such as the daily demand for commodities and a deadline. The deadline is the maximul time  the traveler should spend in his trip from Venice to Milan.


The model aims to determine the optimal delivery route, maximize  the number of parcels delivered while respecting the deadline and minimizing the travel time.


The methodology used to develop this model has different steps. First a formulation of the problem must be generated, this formulation will help define the parameters, the decision variables, the objective function and the constraints for the model to run.


In conclusion, our goal and the final result of the model is to determine the most efficient route between locations that saves travel expenses and ensures that all goods are delivered for a given daily demand using adequate techniques.

# 2 - Acronyms and definitions

- *Distance: The distances used in this project are a result of a mathematical equation that has the latitude and longitude of the two points as parameters. This method assumes that the earth is a sphere, and the distance doesn't take routes and roads in consideration, therefore it is "as the crow flies"*.

- *Time: The time spent by the traveler to go from point A to point B is solely dependant on the driving speed of the traveler, for this project, we assume that the traveler has a constant speed of 60km/h*.

# 3 - Problem statement

Gonzales Company is planning a business trip through the Veneto region in Italy, with the goal of visiting as many cities as possible on their way to Milan. The trip will start in Venice and end in Milan. The company has limited time and wants to maximize the number of parcels delivered while adhering to certain constraints.




# 4 - System

## 4.1 - Agents/DMs
*Decision  Maker:*
- The Courier.

## 4.2 - Assumptions
*Demand:*
- The demand of each city is proportionate to its population
- The Courier's travel speed is constant, wether in the city or in the highway.
- The Earth is a sphere (In order to get the distance from the coordinates)



# Preliminary operations
- First we need to set up the necessary environment for solving the optimization problem using Pyomo.
- We will be getting our data from Drive, therefore colab must be imported
- GLPK must be imported in order to solve the model
- Pandas must be imported to manipulate the data
- distance must be imported to calculate the distance between the coordinates

In [ ]:
# Preliminary operations' code
import pandas as pd
from geopy.distance import distance


working_in_colab = True # Set this variable equal to True if you are working in Colab, False otherwise
if working_in_colab:
    # WARNING: On Colab only
    # This needs to be done once at the start of each session.
    !pip install -q pyomo              # Quiet installation of pyomo using pip.
    !apt-get install -y -qq coinor-cbc # Installation of COIN-OR CBC.
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)   # Mount your drive

if working_in_colab:
    input_directory = "/content/drive/MyDrive/data/italycoord/"
    output_directory = "/content/drive/MyDrive/data/italycoord/"
    cbc_path = "/usr/bin/cbc"
else:
    input_directory = "./Input/"
    output_directory = "./Output/"
    cbc_path = "C:/Program Files/Cbc/bin/cbc.exe" # write the path of the cbc.exe file on your pc

# Preliminary operations' code
input_directory = "./Input/"
output_directory = "./Output/"
cbc_path = "C:/Program Files/Cbc/bin/cbc.exe" # write the path of the cbc.exe file on your pc

#COIN-OR CBC is a multi-threaded open-source Coin-or branch and cut mixed-integer linear programming solver
!apt-get install -y -qq coinor-cbc
#pyomo
!pip install -q pyomo

!apt-get install -y -qq glpk-utils

from pyomo.environ import *


Mounted at /content/drive


# 5 - Data

- The source of the coordinates of the cities is taken from our course's moodle page 'Italycoord.csv'

In [ ]:
import pandas as pd
# code that reads the input data files
#Coordinate distances was taken from the course's moodle
cities = ['Belluno','Feltre',"Cortina d'Ampezzo",'Padova','Abano Terme','Este','Rovigo','Adria','Porto Tolle','Treviso','Conegliano','Montebelluna','Venezia','Chioggia','Mirano','Verona','Bardolino','Peschiera del Garda','Vicenza','Bassano del Grappa','Marostica']

#coords = pd.read_csv('/content/drive/MyDrive/data/italycoord/itacoordcity.csv',delimiter=';', encoding='ISO-8859-1')
coords = pd.read_csv('drive/MyDrive/ItalyCoord.csv',delimiter=';', encoding='ISO-8859-1')

#Cities
coords = coords[coords['comune'].isin(cities)]

# Create an empty distance matrix
n_regions = len(coords)
Data = pd.DataFrame(index=coords['comune'], columns=coords['comune'])
# Calculate the distances between all pairs of regions
for i in range(n_regions):
    for j in range(i+1, n_regions):
        coord1 = (coords.iloc[i]['lat'], coords.iloc[i]['lng'])
        coord2 = (coords.iloc[j]['lat'], coords.iloc[j]['lng'])
        dist = distance(coord1, coord2).km
        Data.iloc[i,j] = dist
        Data.iloc[j,i] = dist

#Dictionnary of distances
distances = {}
for i in range(len(Data)):
  for j in range(len(Data)):
    if j!=i:
      distances[(Data.index[i],Data.columns[j])] = Data.iloc[i,j]
new_dict = {}
for key, value in distances.items():
    if key not in new_dict:
        new_dict[key] = value
#new_dict
# code that defines the dictionaries and/or frames used in the model.

### Time Matrix
We used the distances between each pair of cities to calculate the time spent on the road by the driver to reach a specific city and we stored them under each pair of city


In [ ]:
from geopy.distance import distance
# Initialize an empty dictionary to hold the travel times
travel_times = {}

# Loop over the keys in the 'distances' dictionary
for key in distances.keys():
    # Get the distance between the two cities
    distance = distances[key]

    # Calculate the time taken to travel between the cities at 60 km/h
    time = distance / 60.0  # hours
    time *= 60  # convert to minutes

    # Reverse the order of the cities in the key to get the key for the
    # opposite direction (e.g., ('city1', 'city2') becomes ('city2', 'city1'))
    reverse_key = tuple(reversed(key))

    # Add the travel time to the 'travel_times' dictionary for both directions
    travel_times[key] = time
    travel_times[reverse_key] = time

# Now the 'travel_times' dictionary should have keys in the form ('city1', 'city2')
# and values representing the time taken to travel from 'city1' to 'city2' in minutes


## 5.1 Mathematical  Model

### 5.1.2 - Model

In [ ]:
#Declare the Modelµ
model = AbstractModel()


### 5.1.3 - Sets

*Cities/nodes*

- Each node is a representation of a city visited or waiting to  be visited. These cities have characteristics such as:

*Arcs/edges*

- They link the cities in the graphical representation and specify the time required to reach city j starting from city i.



In [ ]:
model.NODES = Set()
model.ARCS = Set(within = model.NODES*model.NODES)

### 5.1.4 - Parameters

- NetDemand: Created for the sole purpose of ensuring that the Courier starts his journey in a specific city, and ends in a specific city.
- Packages: The demand of each city
- Cost: The time spent by the courier from a city to a different city
- Alpha: The weight the company wishes to determine on the objective function

In [ ]:
#Parameters
model.netDemand = Param(model.NODES) #used to get from veince(-1) to milan(1)
model.packages = Param(model.NODES)
model.cost = Param(model.ARCS)
model.alpha = Param(default=0) # weight of demand in objective function
#alpha == 0 ==> shortest distance

### 5.1.5 - Variables

- u[i]: The position of each city in the tour
- x[i,j]: 1 if the courier goes from city i to city j, 0 otherwise

In [ ]:
# New variables ordr of nds
model.u = Var(model.NODES, within=NonNegativeIntegers)
#Decision Variables sel or no
model.x = Var(model.ARCS , within = Binary)

### 5.1.6 - Constraints and Objective

**Objective Function**: aims to balance the total demand satisfied with the cost of the route.

Let $C$ be the set of cities to be visited, and let $p_j$ be the demand (interchangable with packages) in city $j$. Let $t_{i,j}$ be the travel time between cities $i$ and $j$, and let $T$ be the maximum allowable travel time.

We can then formulate the objective as follows:

\begin{align*}
\text{maximize} \quad & \alpha\sum_{i,j\in C}  x_{i,j}p_j - (1-\alpha)\sum_{i,j\in C} x_{i,j}t_{i,j}
\end{align*}

where $x_{i,j}$ is a binary variable that indicates whether we travel from city $i$ to city $j$ along the route, and $\alpha$ is a weight parameter that balances the importance of demand satisfaction and cost minimization. The objective function aims to maximize the total demand satisfied along the route, while also minimizing the cost of traveling between cities.

<br>

**We want to find a route that starts at city $a$ and ends at city $b$, subject to the following constraints:**

The total travel time does not exceed $T$: $\sum_{i,j\in C} t_{i,j} x_{i,j} \leq T$.<br>

Net Demand Rule: $\sum_{k\in C} x_{k,i} - \sum_{j\in C} x_{i,j} = d_i$ for all $i\in C$

Subtour elimination constraint: $u_{i} - u_{j} + \sum_{i,j\in S} x_{i,j} \leq |S|-1$ for all $S\subset C$, $a\in S$, and $b\notin S$.<br>
The number of cities visited should be at least $n$: $\sum_{i,j\in C} x_{i,j} \geq n$.<br>
The number of cities visited should be at most $m$: $\sum_{i,j\in C} x_{i,j} \leq m$.<br>

Where $C$ is the set of cities, $T$ is the maximum travel time, $p$ is the demand of the city, and the $d$ is the Net Demand of each city. $x_{i,j}$ is a binary decision variable that equals 1 if the route goes from city $i$ to city $j$, and $t_{i,j}$ represent the travel time and the time spent in the trip from city $i$ to city $j$, respectively.

In [ ]:
#Objective Function
def obj_rule(model):
    return (model.alpha * sum(model.x[i,j]*model.packages[j] for (i,j) in model.ARCS)) - ((1 - model.alpha) * sum(model.x[i,j]*model.cost[i,j] for (i,j) in model.ARCS))
model.obj = Objective(rule = obj_rule, sense = maximize)


#Constraints
#He shouldnt stay for more than a particular number of minutes
def deadline_constraint(model):
  T = 360
  return sum(model.x[i,j]*model.cost[i,j] for (i,j) in model.ARCS) <= T
model.deadline_constraint = Constraint(rule=deadline_constraint)



#To make sure that the starting city is venezia and the arrival city is Milano
def net_demand_rule(model,i):
  return (sum(model.x[k,i] for k in model.NODES if (k,i) in model.ARCS)  - sum(model.x[i,j] for j in model.NODES if (i,j) in model.ARCS) == model.netDemand[i])
model.netDemandConstraints = Constraint(model.NODES , rule = net_demand_rule)

# Subtour elimination constraint
def no_subtours_rule(model, i, j):
    if i != j and (i, j) in model.ARCS:
        return model.u[i] - model.u[j] + len(model.NODES)*model.x[i,j] <= len(model.NODES)-1
    else:
        return Constraint.Skip
model.no_subtours = Constraint(model.ARCS, rule=no_subtours_rule)


'''
# Visit both A & B
def visit_A_and_B_rule(model):
  A = 'Marostica'
  B = 'Belluno'
  return sum(model.x[i, A] for i in model.NODES if i!=A) + sum(model.x[i, B] for i in model.NODES if i!=B) == 2
model.visit_A_and_B = Constraint(rule=visit_A_and_B_rule)
'''

# Visit either A or B
def visit_either_A_and_B_rule(model):
  A = 'Feltre'
  B = 'Este'
  return sum(model.x[i, A] for i in model.NODES if i!=A) + sum(model.x[i, B] for i in model.NODES if i!=B) == 1
model.visit_either_A_and_B_rule = Constraint(rule=visit_either_A_and_B_rule)


'''
#If city A is visited, then city B must be visited
def visit_B_if_A_visited(model):
  A = 'Feltre'
  B = 'Este'
  return (sum(model.x[j,B] for j in model.NODES if j!=B) >= sum(model.x[i,A] for i in model.NODES if i!=A))
model.visit_B_if_A_visited = Constraint(rule=visit_B_if_A_visited)'''

'''
#If city A is visited, then city B must NOT be visited
def dont_visit_B_if_A_visited(model):
  A = 'Porto_Tolle'
  B = 'Adria'
  return (sum(model.x[j,B] for j in model.NODES if j!=B) <= 1 - sum(model.x[i,A] for i in model.NODES if i!=A))
model.dont_visit_B_if_A_visited = Constraint(rule=dont_visit_B_if_A_visited)
'''

'''
#if city A is visited, then neither city B nor city C must be visited
def dont_visit_B_and_c_if_A_visited(model):
  A = 'Belluno'
  B = 'Abano_Terme'
  C = 'Este'
  return (sum(model.x[j,B] for j in model.NODES if j!=B) + sum(model.x[k,C] for k in model.NODES if k!=C) <= 2 - 2*sum(model.x[i,A] for i in model.NODES if i!=A))
model.dont_visit_B_and_c_if_A_visited = Constraint(rule=dont_visit_B_and_c_if_A_visited)
#Xic + XiB <= 2 - 2XiA
'''
'''
#n cities must be visited at least
def min_visits(model):
  n = 2
  return sum(model.x[i,j] for (i,j) in model.ARCS) >= n
model.min_visits = Constraint(rule=min_visits)
'''


'''def max_visits(model):
  return sum(model.x[i,j] for (i,j) in model.ARCS) <= 8
model.max_visits = Constraint(rule=max_visits)'''



#These cities must be visited
def cities_to_visit(model):
  subset = ['Belluno']
  l = len(subset)
  return sum(model.x[i,j] for (i,j) in model.ARCS if i!=j and j in subset) >= l
model.cities_to_visit = Constraint(rule=cities_to_visit)


## Model Solution

In [ ]:
#Solving the problem
data = DataPortal()
data.load(filename = '/content/sample_data/Data.dat', model=model)
instance = model.create_instance(data)

optimizer = SolverFactory('glpk', timelimit=1)
optimizer.options['mipgap'] = 0.01
optimizer.solve(instance)
instance.display()
result = {}
for i in instance.NODES:
    for j in instance.NODES:
        if (i,j) in instance.ARCS and instance.x[i,j].value == 1:
            print("From city", i, "to city", j)
            result[i]=j

Model unknown

  Variables:
    u : Size=22, Index=NODES
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
                Abano_Terme :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                      Adria :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                  Bardolino :     0 :   0.0 :  None : False : False : NonNegativeIntegers
         Bassano_del_Grappa :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                    Belluno :     0 :   1.0 :  None : False : False : NonNegativeIntegers
                   Chioggia :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                 Conegliano :     0 :   0.0 :  None : False : False : NonNegativeIntegers
           Cortina_dAmpezzo :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                       Este :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                     Feltre :     0 :   2.0 :  None : Fa

In [ ]:
cities_list = ['Venezia']

# Loop until we reach Milano
while cities_list[-1] != 'Milano':
    # Get the last city in the list
    last_city = cities_list[-1]

    # Find the next city in the dictionary
    next_city = result[last_city]

    # Add the next city to the list
    cities_list.append(next_city)

# Print the final list of cities
for i in cities_list:
  if i !='Milano':
    print(i,' -> ',end='')
  else:
    print(i)

Venezia  -> Belluno  -> Feltre  -> Milano


## 5.3 - Analysis
The model solution is reliable and  and applicable for real world small problems if provided the required data.

It can also be altered in a way that the constraints are difference with accordance to different scenarios.
For example we can impose the last city visited.
We can also allow the driver to visit a city more than once.
We can omit the penalty cost and add a bonus if parcels are delivered prior to their estimated time in an excellent state.